In [ ]:
"""
The Python script "gene_ID_and_off_gene_symbol_check.py" has been
successfully run on the Hemera HPC cluster. However, on throwing a
glance at the output file and subsequently scutinising the
"ID_manufacturer" column in the CSV file, several issues emerged.

The first is that the database query failed for four gene IDs, namely
644862, 441848, 441931 and 441860.

The second is that for some perturbation agents, be they siRNA, pooled
siRNA or esiRNA, more than one target gene is listed. In both the column
"ID_manufacturer" and "Name", the individual entries are separated by
semicolons.

The third is that for some perturbation agents, the entry in both
"ID_manufacturer" and "Name" is "Not available". However, as both the
total amount of perturbation agents this applies to is manageable and
the catalogue number of along with the manufacturer is provided, the
target genes are manually looked up.

As to the first two issues, however, postprocessing is accomplished in
an automated manner.

Apart from that, some records have been discontinued in the NCBI
database, such as the record corresponding to the gene ID 441848. Such
discontinued records contain the sentence "This record was
discontinued". It is decided at a later time how discontinued records
are dealt with.
"""

In [1]:
import string

import numpy as np
import pandas as pd

In [2]:
# Load the screen data
# Bear in mind that for certain columns, the data type has to be
# manually specified
dtype_dict = {
    "Ensembl_ID_OnTarget_Ensembl_GRCh38_release_87": str,
    "Ensembl_ID_OnTarget_NCBI_HeLa_phs000643_v3_p1_c1_HMB": str,
    "Gene_Description": str,
    "ID": str,
    "ID_OnTarget_Ensembl_GRCh38_release_87": str,
    "ID_OnTarget_Merge": str,
    "ID_OnTarget_NCBI_HeLa_phs000643_v3_p1_c1_HMB": str,
    "ID_OnTarget_RefSeq_20170215": str,
    "ID_manufacturer": str,
    "Name_alternatives": str,
    "PLATE_QUALITY_DESCRIPTION": str,
    "RefSeq_ID_OnTarget_RefSeq_20170215": str,
    "Seed_sequence_common": str,
    "WELL_QUALITY_DESCRIPTION": str,
    "siRNA_error": str,
    "siRNA_number": str,
    "Precursor_Name": str
}

# Dask DataFrames exhibit a peculiarity regarding the index labels: By
# default, the index labels are integers, just as with Pandas
# DataFrames. However, unlike Pandas DataFrames, the index labels do not
# monotonically increase from 0, but restart at 0 for each partition,
# thereby resulting in duplicated index labels (Dask subdivides a Dask
# DataFram into multiple so-called partitions as the whole idea behind
# Dask is to handle large data sets in a memory-efficient way, https://
# docs.dask.org/en/stable/generated/dask.dataframe.DataFrame.reset_
# index.html)
# Hence, performing operations with Dask DataFrames might potentially
# raise the `ValueError: cannot reindex on an axis with duplicate
# labels` error
# In this case, loading the entire data set into a Pandas DataFrame is
# feasible, which is why this is preferred to loading it into a Dask
# DataFrame (strangely enough, this has not been possible in the very
# beginning, which is why Dask was used in the first place)
main_csv_df = pd.read_csv(
    "VacciniaReport_20170223-0958_ZScored_conc_and_NaN_adjusted.csv",
    sep="\t",
    dtype=dtype_dict
)

# Bear in mind that due to operator precedence, i.e. "|" (logical OR)
# having precedence over equality checks, the equality checks have to be
# surrounded by parentheses
single_pooled_siRNA_and_esiRNA_df = main_csv_df.loc[
    (main_csv_df["WellType"] == "SIRNA")
    |
    (main_csv_df["WellType"] == "POOLED_SIRNA")
    |
    (main_csv_df["WellType"] == "ESIRNA")
]

In [3]:
# Determine the amount of perturbation agents for which the target genes
# are not specified
ID_manufacturer_series = single_pooled_siRNA_and_esiRNA_df[
    "ID_manufacturer"
]

n_target_not_specified = np.count_nonzero(
    ID_manufacturer_series == "Not available"
)

print(
    "Amount of perturbation agents for which the target genes are not "
    f"specified: {n_target_not_specified}"
)

Amount of perturbation agents for which the target genes are not specified: 22


In [5]:
# Now, address the first two issues in an automated manner (failed
# database query for four gene IDs and the listing of several target
# genes separated by semicolons)
# To this end, the CSV file with the updated gene IDs and official gene
# symbols is also loaded
# Unfortunately, when saving the updated Pandas DataFrame to a CSV file,
# the separator has not been specified, which is why the default
# separator has been used
# This, however, conflicts with the usage of commata in some entries
# Thus, prior to loading the updated CSV file, the commata have to be
# replaced with tab stops in a sophisticated manner taking account of
# this difference between actual delimiters and commata which are part
# of entries
# To this end, columns containing commata in their entries have to be
# identified
feature_names = main_csv_df.columns
features_with_commata = []
features_with_commata_indices = []

for i, feature_name in enumerate(feature_names):
    feature_series = main_csv_df[feature_name]
    if feature_series.dtype != "object":
        continue
    # Bear in mind that in order to check for the presence of a
    # substring in a Pandas DataFrame, "pandas.Series.str.contains" has
    # to be used rather than "pandas.Series.isin" as the latter only
    # verifies complete matches between column entries and query strings
    n_commata_in_entries = feature_series.str.contains(",").sum()
    if n_commata_in_entries > 0:
        features_with_commata.append(feature_name)
        features_with_commata_indices.append(i)

# Excel uses upper case letters instead of numbers in order to index
# columns
# Hence, for the sake of convenience, the numeric indices are
# simultaneously mapped to the corresponding alphabetical indices
# The built-in string module allows to fetch a string representing the
# entire alphabet
alphabet_list = list(string.ascii_uppercase)

alphabetic_indices_list = list(string.ascii_uppercase)
for first_letter in alphabet_list[:3]:
    for second_letter in alphabet_list:
        alphabetic_indices_list.append(first_letter + second_letter)

numeric_alphabetic_index_dict = {}
for numeric_index, alphabetic_index in enumerate(alphabetic_indices_list):
    numeric_alphabetic_index_dict[numeric_index] = alphabetic_index

max_feature_name_length = max(map(len, features_with_commata))

print(
    "The following features/columns contain commata in their entries:\n",
    "Feature name".ljust(max_feature_name_length + 1),
    "Numeric index".ljust(14),
    "Alphabetical Index\n",
    "-" * (max_feature_name_length + 1 + 14 + len("Alphabetical index")),
    sep="",
    end=""
)

for i, feature_name in zip(
    features_with_commata_indices, features_with_commata
):
    print(
        "\n",
        feature_name.ljust(max_feature_name_length + 1),
        str(i).ljust(14),
        numeric_alphabetic_index_dict[i],
        sep="",
        end=""
    )

The following features/columns contain commata in their entries:
Feature name      Numeric index Alphabetical Index
--------------------------------------------------
Name_alternatives 30            AE
Gene_Description  62            BK

In [6]:
# Now, the entries of the columns immediately following those the
# entries of which contain commata are scrutinised
# Ideally, they exhibit common characteristics that can be leveraged for
# the distinction between actual delimiters and commata belonging to
# entries
following_series_1 = main_csv_df[feature_names[31]]
following_series_2 = main_csv_df[feature_names[63]]

unique_vals_series_1 = np.unique(following_series_1)
unique_vals_series_2 = np.unique(following_series_2)

print(unique_vals_series_1)
print(unique_vals_series_2)

['MultipleTargets' 'NoTargets' 'Not available' 'OK' 'POOLED_SIRNA_ERROR'
 'TargetMismatch' 'Unknown']
['ENST00000000233;ENST00000463733'
 'ENST00000000233;ENST00000463733;ENST00000415666;ENST00000467281;ENST00000489673;ENST00000459680'
 'ENST00000000233;ENST00000463733;ENST00000415666;ENST00000489673;ENST00000464403'
 ... 'ENST00000515849;ENST00000302763;ENST00000355078' 'ENST00000516084'
 'Not available']


In [9]:
# Regarding the second of the two investigated columns, it emerges that
# the vast majority of its entries begin with the sequence "ENST"
# It is investigated whether this is indeed the case for all entries or
# whether there are some exceptions
print(all([entry[:4] == "ENST" for entry in unique_vals_series_2]))

False


In [10]:
# Apparently, there are entries not starting with the "ENST" sequence
# They are extracted and examined
# The arguably easiest way to accomplish this is boolean indexing, which
# is provided by NumPy
outcast_vals_arr = unique_vals_series_2[
    [entry[:4] != "ENST" for entry in unique_vals_series_2]
]
print(outcast_vals_arr)

['Not available']


In [11]:
# It becomes apparent that I am damn lucky as I indeed am able to
# leverage common characterics in order to distinguish actual delimiters
# from commata belonging to entries
with open(
    "Vaccinia_Report_NCBI_Gene_IDs_and_official_gene_symbols_updated.csv",
    "r"
) as f:
    file_lines = f.readlines()

In [17]:
# List comprising unique values of column 31, i.e. the column following
# column 30 ("Name_alternatives")
siRNA_error_options = [
    "MultipleTargets",
    "NoTargets",
    "Not available",
    "OK",
    "POOLED_SIRNA_ERROR",
    "TargetMismatch",
    "Unknown"
]

# Iterate through the lines, modify them accordingly and write the
# adjusted lines to a new output file
with open("adjusted_file.csv", "w") as f:
    for line in file_lines:
        # When employing the built-in split method for strings, the
        # separation character is not retained, but discarded
        # Hence, by employing a trick involving a nested list
        # comprehension, the separation character is added at its
        # corresponding positions
        # (https://www.geeksforgeeks.org/python-string-split-including-spaces/)
        split_line = [i for j in line.split(",") for i in (j, ",")][:-1]
        
        line_comma_indices = [
            i for i, x in enumerate(split_line) if x == ","
        ]

        # Determine the indices of commata belonging to entries in lieu
        # of being delimiters
        entry_commata_list = []

        # First, deal with column 30, i.e. "Name alternatives"
        # Keep in mind that it is iterated through the list
        # `split_line`, which encompasses both the entries as well as
        # commata
        # Therefore, the index corresponding to column 30 is not 30, but
        # 1 + 28 * 2 + 1 = 58 (counting starts with 0, hence the comma
        # of the first entry has index 1; to account for the remaining
        # 28 entries, 28 * 2 is added; finally, in order to obtain the
        # index of the 30th entry, 1 is added)
        entry_index_1 = 58
        subsequent_entry = split_line[entry_index_1]
        while subsequent_entry not in siRNA_error_options:
            entry_commata_list.append(entry_index_1 - 1)
            entry_index_1 += 2
        
        # Now, do the same thing with column 62, i.e.
        # ("Gene_Description")
        # Again, the index of the entry in `split_line` corresponding to
        # column 62 is not 62, but 1 + 60 * 2 + 1 = 122
        # Note that the index of the first entry to investigate has to
        # be adjusted according to the previous amount of "entry
        # commata"
        entry_index_2 = 122 + len(entry_commata_list) * 2
        subsequent_entry = split_line[entry_index_2]
        while ...:
            pass
        
        n_additional_commata = 0
        for comma_index in line_comma_indices:
            # Due to the fact that counting begins with 0, the amount of
            # delimiters preceding a column with a certain index equals
            # that index, at least before the occurrence of additional
            # commata in the entries
            # Thus, the first column containing commata in its entries
            # is preceded by 30 delimiters
            if comma_index < 31:
                split_line[comma_index] = "\t"
            # Now, a sophisticated approach is required in order to
            # distinguish commata belonging to entries from actual
            # delimiters
            elif ...:
                pass
        break

[1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63, 65, 67, 69, 71, 73, 75, 77, 79, 81, 83, 85, 87, 89, 91, 93, 95, 97, 99, 101, 103, 105, 107, 109, 111, 113, 115, 117, 119, 121, 123, 125, 127, 129, 131, 133, 135, 137, 139, 141, 143, 145, 147, 149, 151, 153, 155, 157, 159, 161, 163, 165, 167, 169, 171, 173, 175, 177, 179, 181]


In [20]:
print(1 + 60 * 2 + 1)

122


In [13]:
print(file_lines[0])

Barcode,PlateType,PLATE_QUALITY_STATUS,PLATE_QUALITY_DESCRIPTION,BEE.RELEASE.STATUS,PLATE_TYPE,BATCH,Space,Group,Experiment,ExperimentType,Library,PATHOGEN,GENESET,REPLICATE,WellRow,WellColumn,WellName,WellType,WELL_QUALITY_DESCRIPTION,WELL_QUALITY_STATUS,PublicationLink_well,PublicationLink_material,Manufacturer,Catalog_number,ID,ID_manufacturer,ID_String,ID_openBIS,Name,Name_alternatives,siRNA_error,Seed_sequence_antisense_5_3,Seed_sequence_sense_5_3,Sequence_antisense_5_3,Sequence_sense_5_3,Sequence_target_sense_5_3,siRNA_number,Seed_sequence_common,Concentration [pmol],Seeded_cell_number,eCount_oCells,dArea_oNuclei,dArea_oPeriNuclei,dArea_oCells,dArea_oVoronoiCells,dIntensity_cPathogen_eMean_oNuclei,dIntensity_cPathogen_eMean_oPeriNuclei,dIntensity_cPathogen_eMean_oCells,dIntensity_cPathogen_eMean_oVoronoiCells,dIntensity_cLatePathogen_eMean_oNuclei,dIntensity_cLatePathogen_eMean_oPeriNuclei,dIntensity_cLatePathogen_eMean_oCells,dIntensity_cLatePathogen_eMean_oVoronoiCells,dInfecti

In [14]:
with open("VacciniaReport_20170223-0958_ZScored_conc_and_NaN_adjusted.csv", "r") as f:
    boring_lines = f.readlines()

print(boring_lines[0])

Barcode	PlateType	PLATE_QUALITY_STATUS	PLATE_QUALITY_DESCRIPTION	BEE.RELEASE.STATUS	PLATE_TYPE	BATCH	Space	Group	Experiment	ExperimentType	Library	PATHOGEN	GENESET	REPLICATE	WellRow	WellColumn	WellName	WellType	WELL_QUALITY_DESCRIPTION	WELL_QUALITY_STATUS	PublicationLink_well	PublicationLink_material	Manufacturer	Catalog_number	ID	ID_manufacturer	ID_String	ID_openBIS	Name	Name_alternatives	siRNA_error	Seed_sequence_antisense_5_3	Seed_sequence_sense_5_3	Sequence_antisense_5_3	Sequence_sense_5_3	Sequence_target_sense_5_3	siRNA_number	Seed_sequence_common	Concentration [pmol]	Seeded_cell_number	eCount_oCells	dArea_oNuclei	dArea_oPeriNuclei	dArea_oCells	dArea_oVoronoiCells	dIntensity_cPathogen_eMean_oNuclei	dIntensity_cPathogen_eMean_oPeriNuclei	dIntensity_cPathogen_eMean_oCells	dIntensity_cPathogen_eMean_oVoronoiCells	dIntensity_cLatePathogen_eMean_oNuclei	dIntensity_cLatePathogen_eMean_oPeriNuclei	dIntensity_cLatePathogen_eMean_oCells	dIntensity_cLatePathogen_eMean_oVoronoiCells	dInfecti

In [15]:
print("Deine\tMutter")

Deine	Mutter


In [5]:
updated_main_csv_df = pd.read_csv(
    "Vaccinia_Report_NCBI_Gene_IDs_and_official_gene_symbols_updated.csv",
    sep="\t",
    dtype=dtype_dict
)

  Barcode,PlateType,PLATE_QUALITY_STATUS,PLATE_QUALITY_DESCRIPTION,BEE.RELEASE.STATUS,PLATE_TYPE,BATCH,Space,Group,Experiment,ExperimentType,Library,PATHOGEN,GENESET,REPLICATE,WellRow,WellColumn,WellName,WellType,WELL_QUALITY_DESCRIPTION,WELL_QUALITY_STATUS,PublicationLink_well,PublicationLink_material,Manufacturer,Catalog_number,ID,ID_manufacturer,ID_String,ID_openBIS,Name,Name_alternatives,siRNA_error,Seed_sequence_antisense_5_3,Seed_sequence_sense_5_3,Sequence_antisense_5_3,Sequence_sense_5_3,Sequence_target_sense_5_3,siRNA_number,Seed_sequence_common,Concentration [pmol],Seeded_cell_number,eCount_oCells,dArea_oNuclei,dArea_oPeriNuclei,dArea_oCells,dArea_oVoronoiCells,dIntensity_cPathogen_eMean_oNuclei,dIntensity_cPathogen_eMean_oPeriNuclei,dIntensity_cPathogen_eMean_oCells,dIntensity_cPathogen_eMean_oVoronoiCells,dIntensity_cLatePathogen_eMean_oNuclei,dIntensity_cLatePathogen_eMean_oPeriNuclei,dIntensity_cLatePathogen_eMean_oCells,dIntensity_cLatePathogen_eMean_oVoronoiCells,dInfec